# Pyomo – Optimization Modeling in Python

In [2]:
# To Install Uncomment if required
!pip install pyomo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 7.3 MB/s eta 0:00:00


## Concrete Model Examples
min x1 +2x2
s.t. 3x1 +4x2 ≥ 1
2x1 +5x2 ≥ 2
x1, x2 ≥ 0

In [3]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()
model.x_1 = pyo.Var(within=pyo.NonNegativeReals)
model.x_2 = pyo.Var(within=pyo.NonNegativeReals)
model.obj = pyo.Objective(expr=model.x_1 + 2 * model.x_2)
model.con1 = pyo.Constraint(expr=3 * model.x_1 + 4 * model.x_2 >= 1)
model.con2 = pyo.Constraint(expr=2 * model.x_1 + 5 * model.x_2 >= 2)

In [4]:
# Data

N = [1, 2]
M = [1, 2]
c = {1: 1, 2: 2}
a = {(1, 1): 3, (1, 2): 4, (2, 1): 2, (2, 2): 5}
b = {1: 1, 2: 2}


In [ ]:
# This LP can be expressed with a concrete model in Pyomo

model = pyo.ConcreteModel()
model.x = pyo.Var(N, within=pyo.NonNegativeReals)

def obj_rule(model):
    return sum(c[i] * model.x[i] for i in N)

model.obj = pyo.Objective(rule=obj_rule)

def con_rule(model, m):
    return sum(a[m, i] * model.x[i] for i in N) >= b[m]

model.con = pyo.Constraint(M, rule=con_rule)

In [5]:
# Abstract Model Examples


## Abstract Model Examples




In [7]:
# Same LP can also be viewed as an abstract mathematical model

model = pyo.AbstractModel()

model.N = pyo.Set()
model.M = pyo.Set()
model.c = pyo.Param(model.N)
model.a = pyo.Param(model.M, model.N)
model.b = pyo.Param(model.M)

model.x = pyo.Var(model.N, within=pyo.NonNegativeReals)


def obj_rule(model):
    return sum(model.c[i] * model.x[i] for i in model.N)


model.obj = pyo.Objective(rule=obj_rule)


def con_rule(model, m):
    return sum(model.a[m, i] * model.x[i] for i in model.N) >= model.b[m]


model.con = pyo.Constraint(model.M, rule=con_rule)

## Graph Coloring Example
Adapted from Jonathan L. Gross and Jay Yellen, "Graph Theory and Its Applications, 2nd Edition"

In [8]:
# Define data for the graph of interest.
vertices = set(
    ['Ar', 'Bo', 'Br', 'Ch', 'Co', 'Ec', 'FG', 'Gu', 'Pa', 'Pe', 'Su', 'Ur', 'Ve']
)

edges = set(
    [
        ('FG', 'Su'),
        ('FG', 'Br'),
        ('Su', 'Gu'),
        ('Su', 'Br'),
        ('Gu', 'Ve'),
        ('Gu', 'Br'),
        ('Ve', 'Co'),
        ('Ve', 'Br'),
        ('Co', 'Ec'),
        ('Co', 'Pe'),
        ('Co', 'Br'),
        ('Ec', 'Pe'),
        ('Pe', 'Ch'),
        ('Pe', 'Bo'),
        ('Pe', 'Br'),
        ('Ch', 'Ar'),
        ('Ch', 'Bo'),
        ('Ar', 'Ur'),
        ('Ar', 'Br'),
        ('Ar', 'Pa'),
        ('Ar', 'Bo'),
        ('Ur', 'Br'),
        ('Bo', 'Pa'),
        ('Bo', 'Br'),
        ('Pa', 'Br'),
    ]
)

ncolors = 4
colors = range(1, ncolors + 1)


# Python import statement
import pyomo.environ as pyo

# Create a Pyomo model object
model = pyo.ConcreteModel()

# Define model variables
model.x = pyo.Var(vertices, colors, within=pyo.Binary)
model.y = pyo.Var()

# Each node is colored with one color
model.node_coloring = pyo.ConstraintList()
for v in vertices:
    model.node_coloring.add(sum(model.x[v, c] for c in colors) == 1)

# Nodes that share an edge cannot be colored the same
model.edge_coloring = pyo.ConstraintList()
for v, w in edges:
    for c in colors:
        model.edge_coloring.add(model.x[v, c] + model.x[w, c] <= 1)

# Provide a lower bound on the minimum number of colors
# that are needed
model.min_coloring = pyo.ConstraintList()
for v in vertices:
    for c in colors:
        model.min_coloring.add(model.y >= c * model.x[v, c])

# Minimize the number of colors that are needed
model.obj = pyo.Objective(expr=model.y)